### Notebook para transformação dos arquivos csv

In [1]:
import pandas as pd

#### Transforma o arquivo dos dados do auxilio

In [2]:
path_auxilio = '../datasets/dados.csv'

df_auxilio = pd.read_csv(path_auxilio, sep=',', encoding= 'UTF-8')

df_auxilio.head()

,codigo_ibge,anomes_s,pab_qtd_fam_benef_i,pab_valor_pago_d,pab_valor_medio_d,pab_extraordinario_qtd_fam_i,pab_extraordinario_valor_pago_d,pab_extraordinario_valor_medio_d
0,110001,202302,1766,1062180.0,601.46,NaN,NaN,NaN
1,110002,202302,5744,3454070.0,601.34,NaN,NaN,NaN
2,110003,202302,301,179593.0,596.65,NaN,NaN,NaN
3,110004,202302,5753,3475545.0,604.13,NaN,NaN,NaN
4,110005,202302,997,599000.0,600.80,NaN,NaN,NaN


In [3]:
df_auxilio = df_auxilio.drop(['pab_extraordinario_qtd_fam_i','pab_extraordinario_valor_pago_d','pab_extraordinario_valor_medio_d'], axis = 1)#remove colunas que não usaremos

df_auxilio['cod_uf'] = df_auxilio['codigo_ibge'].astype(str).str.slice(0, 2)#extrai apenas os 2 primeiros digitos do código ibge (se torna código UF)
df_auxilio = df_auxilio.rename(columns={"anomes_s":"dt_referencia","pab_qtd_fam_benef_i": "qnt_beneficios", "pab_valor_pago_d":"valor_pago", "pab_valor_medio_d":"valor_medio"})

df_auxilio['dt_referencia'] = df_auxilio['dt_referencia'].astype(str)#transforma o campo em string
df_auxilio['dt_referencia'] = pd.to_datetime(df_auxilio['dt_referencia'] + '01', format='%Y%m%d')#transforma data no padrão y-m-d

df_auxilio.head()

,codigo_ibge,dt_referencia,qnt_beneficios,valor_pago,valor_medio,cod_uf
0,110001,2023-02-01,1766,1062180.0,601.46,11
1,110002,2023-02-01,5744,3454070.0,601.34,11
2,110003,2023-02-01,301,179593.0,596.65,11
3,110004,2023-02-01,5753,3475545.0,604.13,11
4,110005,2023-02-01,997,599000.0,600.80,11


#### Transforma o arquivo dos dados do ibge

In [4]:
path_ibge = '../datasets/cidades_ibge.csv'

df_ibge = pd.read_csv(path_ibge, sep=',', encoding= 'Windows 1252', skiprows=6)

df_ibge.head()

,UF,Nome_UF,Região Geográfica Intermediária,Nome Região Geográfica Intermediária,Região Geográfica Imediata,Nome Região Geográfica Imediata,Mesorregião Geográfica,Nome_Mesorregião,Microrregião Geográfica,Nome_Microrregião,Município,Código Município Completo,Nome_Município
0,11,Rondônia,1102,Ji-Paraná,110005,Cacoal,2,Leste Rondoniense,6,Cacoal,15,1100015,Alta Floresta D'Oeste
1,11,Rondônia,1102,Ji-Paraná,110005,Cacoal,2,Leste Rondoniense,6,Cacoal,379,1100379,Alto Alegre dos Parecis
2,11,Rondônia,1101,Porto Velho,110002,Ariquemes,2,Leste Rondoniense,3,Ariquemes,403,1100403,Alto Paraíso
3,11,Rondônia,1102,Ji-Paraná,110004,Ji-Paraná,2,Leste Rondoniense,5,Alvorada D'Oeste,346,1100346,Alvorada D'Oeste
4,11,Rondônia,1101,Porto Velho,110002,Ariquemes,2,Leste Rondoniense,3,Ariquemes,23,1100023,Ariquemes


In [5]:

df_dados_ibge = df_ibge[['Nome_UF', 'UF']]#seleciona as colunas desejadas
df_dados_ibge = df_dados_ibge.rename(columns={"UF": "cod_uf", "Nome_UF":"nm_estado"}).astype(str)#renomeia a coluna de UF para código UF


df_dados_ibge = df_dados_ibge.drop_duplicates()#remove dados duplicados
df_dados_ibge.head()

,nm_estado,cod_uf
0,Rondônia,11
52,Acre,12
74,Amazonas,13
136,Roraima,14
151,Pará,15


### Juntando os arquivos e realizando as últimas transformações

In [6]:
df_result = pd.merge(df_auxilio, df_dados_ibge, on='cod_uf', how='left')#left join entre os arquivos para recuperar o código e o nome do estado
df_result

,codigo_ibge,dt_referencia,qnt_beneficios,valor_pago,valor_medio,cod_uf,nm_estado
0,110001,2023-02-01,1766,1062180.0,601.46,11,Rondônia
1,110002,2023-02-01,5744,3454070.0,601.34,11,Rondônia
2,110003,2023-02-01,301,179593.0,596.65,11,Rondônia
3,110004,2023-02-01,5753,3475545.0,604.13,11,Rondônia
4,110005,2023-02-01,997,599000.0,600.80,11,Rondônia
...,...,...,...,...,...,...,...
89115,522200,2021-11-01,375,77127.0,205.67,52,Goiás
89116,522205,2021-11-01,701,129055.0,184.10,52,Goiás
89117,522220,2021-11-01,533,105392.0,197.73,52,Goiás
89118,522230,2021-11-01,538,96102.0,178.63,52,Goiás


In [9]:
df_result.to_csv('../arquivo_final/data_gold.csv')